In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, auc,roc_curve,accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve,roc_auc_score,brier_score_loss
import time
from lightgbm import LGBMClassifier
import lightgbm as lgb
import pickle
import random
random.seed(37)
np.random.seed(37)
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

import duckdb
# to start an in-memory database
con = duckdb.connect(database = ":memory:")

### ICU close to Admission

1. Check ICU location_category between admission_dttmtime and 48hr stop from admission
2. Check ICU stay at least 24 hr (for ICU - OR - ICU including OR in ICU stay 24hr)

In [ ]:
tables_location="C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main"

race_map = {
    'White': 'White',
    'Black or African American': 'Black',
    'Asian': 'Asian',
    'Other': 'Others',
    'Unknown': 'Others',
    'Did Not Encounter': 'Others',
    'Refusal': 'Others',
    'American Indian or Alaska Native': 'Others',
    'Native Hawaiian or Other Pacific Islander': 'Others',
    np.nan: 'Others'
}

ethnicity_map = {
    'Not Hispanic or Latino': 'Not Hispanic or Latino',
    'Hispanic or Latino': 'Hispanic or Latino',
    'Did Not Encounter': 'Not Hispanic or Latino',
    'Refusal': 'Not Hispanic or Latino',
    '*Unspecified': 'Not Hispanic or Latino',
    np.nan: 'Not Hispanic or Latino'
}

site_name='RUSH'

no_show=['encounter_id', 'age']

In [ ]:
location = pd.read_csv("C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_adt.csv")
encounter = pd.read_csv("C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_encounter_demographics_dispo_clean.csv")
limited=pd.read_csv("C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_limited_identifiers.csv")
demog=pd.read_csv("C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_patient_demographics.csv")

In [ ]:
join=pd.merge(location[['encounter_id','location_category','in_dttm','out_dttm']],\
              limited[['encounter_id','admission_dttm']], on=['encounter_id'], how='left')

icu_data=pd.merge(join,\
                  encounter[['encounter_id','age_at_admission','disposition']], on=['encounter_id'], how='left')

icu_data['in_dttm'] = pd.to_datetime(icu_data['in_dttm'])
icu_data['admission_dttm'] = pd.to_datetime(icu_data['admission_dttm'])
icu_data['out_dttm'] = pd.to_datetime(icu_data['out_dttm'])
#icu_data['age_at_admission'] = icu_data['age_at_admission'].astype(int)

icu_48hr_check = icu_data[
    (icu_data['location_category'] == 'ICU') &
    (icu_data['in_dttm'] >= icu_data['admission_dttm']) &
    (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=48)) & 
    (icu_data['age_at_admission'] >= 18) & (icu_data['age_at_admission'].notna())  
]['encounter_id'].unique()

icu_data=icu_data[icu_data['encounter_id'].isin(icu_48hr_check) & (icu_data['in_dttm'] <= icu_data['admission_dttm'] + pd.Timedelta(hours=72))].reset_index(drop=True)

icu_data = icu_data.sort_values(by=['in_dttm']).reset_index(drop=True)

icu_data["RANK"]=icu_data.sort_values(by=['in_dttm'], ascending=True).groupby("encounter_id")["in_dttm"].rank(method="first", ascending=True).astype(int)

min_icu=icu_data[icu_data['location_category'] == 'ICU'].groupby('encounter_id')['RANK'].min()
icu_data=pd.merge(icu_data, pd.DataFrame(zip(min_icu.index, min_icu.values), columns=['encounter_id', 'min_icu']), on='encounter_id', how='left')
icu_data=icu_data[icu_data['RANK']>=icu_data['min_icu']].reset_index(drop=True)

icu_data.loc[icu_data['location_category'] == 'OR', 'location_category'] = 'ICU'

icu_data['group_id'] = (icu_data.groupby('encounter_id')['location_category'].shift() != icu_data['location_category']).astype(int)
icu_data['group_id'] = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby('encounter_id')['group_id'].cumsum()

icu_data = icu_data.sort_values(by=['in_dttm'], ascending=True).groupby(['encounter_id', 'location_category', 'group_id']).agg(
    min_in_dttm=('in_dttm', 'min'),
    max_out_dttm=('out_dttm', 'max'),
    admission_dttm=('admission_dttm', 'first'),
    age=('age_at_admission', 'first'),
    dispo=('disposition', 'first')
).reset_index()

min_icu=icu_data[icu_data['location_category'] == 'ICU'].groupby('encounter_id')['group_id'].min()
icu_data=pd.merge(icu_data, pd.DataFrame(zip(min_icu.index, min_icu.values), columns=['encounter_id', 'min_icu']), on='encounter_id', how='left')

icu_data=icu_data[(icu_data['min_icu']==icu_data['group_id']) &
         (icu_data['max_out_dttm']-icu_data['min_in_dttm'] >= pd.Timedelta(hours=24))
         ].reset_index(drop=True)

icu_data['after_24hr']=icu_data['min_in_dttm'] + pd.Timedelta(hours=24)

icu_data=icu_data[['encounter_id','min_in_dttm','max_out_dttm','admission_dttm','after_24hr','age','dispo']]

icu_data=pd.merge(icu_data,\
                  demog, on=['encounter_id'], how='left')[['encounter_id','min_in_dttm','after_24hr','admission_dttm','max_out_dttm','age','dispo','sex','ethnicity','race']]
icu_data=icu_data[~icu_data['sex'].isna()].reset_index(drop=True)
icu_data['isfemale']=(icu_data['sex'].str.lower() == 'female').astype(int)
icu_data['isdeathdispo'] = (icu_data['dispo'].str.contains('dead|expired', case=False, regex=True)).astype(int)

icu_data['ethnicity'] = icu_data['ethnicity'].map(ethnicity_map)
icu_data['race'] = icu_data['race'].map(race_map)
icu_data['ICU_stay_hrs']= (icu_data['max_out_dttm'] - icu_data['min_in_dttm']).dt.total_seconds() / 3600

del location,encounter,limited,demog

### Vitals

In [ ]:
vitals = con.execute('''
    SELECT 
        encounter_id,
        CAST(recorded_dttm AS datetime) AS recorded_dttm,
        CAST(vital_value AS float) AS vital_value,
        vital_category  
    FROM 
        read_csv_auto('C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_vitals_clean.csv')
    WHERE 
        vital_category  IN ('weight_kg', 'pulse', 'sbp', 'dbp', 'temp_c','height_inches') 
        AND encounter_id IN (SELECT DISTINCT encounter_id FROM icu_data);
''').df()

vitals=con.execute('''
PIVOT vitals
ON vital_category 
USING first(vital_value)
GROUP BY encounter_id,recorded_dttm;
''').df()

vitals['height_meters'] = vitals['height_inches'] * 0.0254
vitals['bmi'] = vitals['weight_kg'] / (vitals['height_meters'] ** 2)


icu_data_agg=pd.merge(icu_data,vitals, on=['encounter_id'], how='left')
icu_data_agg=icu_data_agg[(icu_data_agg['recorded_dttm'] >= icu_data_agg['min_in_dttm']) & (icu_data_agg['recorded_dttm'] <= icu_data_agg['after_24hr'])].reset_index(drop=True)

icu_data_agg = icu_data_agg.groupby(['encounter_id']).agg(
    min_bmi=('bmi', 'min'),
    max_bmi=('bmi', 'max'),
    avg_bmi=('bmi', 'mean'),
    min_weight_kg=('weight_kg', 'min'),
    max_weight_kg=('weight_kg', 'max'),
    avg_weight_kg=('weight_kg', 'mean'),
    min_pulse=('pulse', 'min'),
    max_pulse=('pulse', 'max'),
    avg_pulse=('pulse', 'mean'),
    min_sbp=('sbp', 'min'),
    max_sbp=('sbp', 'max'),
    avg_sbp=('sbp', 'mean'),
    min_dbp=('dbp', 'min'),
    max_dbp=('dbp', 'max'),
    avg_dbp=('dbp', 'mean'),
    min_temp_c=('temp_c', 'min'),
    max_temp_c=('temp_c', 'max'),
    avg_temp_c=('temp_c', 'mean'),
).reset_index()

icu_data=pd.merge(icu_data,icu_data_agg, on=['encounter_id'], how='left')

del vitals,icu_data_agg

In [ ]:
icu_data.drop(columns=no_show).head()

### Labs

In [ ]:
labs = con.execute('''
    SELECT 
        encounter_id,
        CAST(lab_order_dttm AS datetime) AS lab_order_dttm,
        TRY_CAST(lab_value AS float) AS lab_value,
        lab_category 
    FROM 
        read_csv_auto('C:/Users/vchaudha/OneDrive - rush.edu/CLIF-1.0-main/rclif/clif_labs_clean.csv')
    WHERE 
        ((lab_category='monocyte' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_category='lymphocyte' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_category='basophil' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_category='neutrophil' and reference_unit = '%' and lab_type_name='standard') OR
        (lab_category='albumin' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_category='ast' and reference_unit = 'U/L' and lab_type_name='standard') OR
        (lab_category='total_protein' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_category='alkaline_phosphatase' and reference_unit = 'U/L' and lab_type_name='standard') OR
        (lab_category='bilirubin_total' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_category='bilirubin_conjugated' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_category='calcium' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_category='chloride' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_category='potassium' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_category='sodium' and reference_unit = 'mmol/L' and lab_type_name='standard') OR
        (lab_category='glucose_serum' and reference_unit = 'mg/dL' and lab_type_name='standard') OR
        (lab_category='hemoglobin' and reference_unit = 'g/dL' and lab_type_name='standard') OR
        (lab_category='platelet count' and reference_unit = 'K/uL' and lab_type_name='standard') OR
        (lab_category='wbc' and reference_unit = 'K/uL' and lab_type_name='standard'))  
        AND encounter_id IN (SELECT DISTINCT encounter_id FROM icu_data);
''').df()

labs=con.execute('''
PIVOT labs
ON lab_category
USING first(lab_value)
GROUP BY encounter_id,lab_order_dttm;
''').df()

In [ ]:

icu_data_agg=pd.merge(icu_data,labs, on=['encounter_id'], how='left')
icu_data_agg=icu_data_agg[(icu_data_agg['lab_order_dttm'] >= icu_data_agg['min_in_dttm']) & (icu_data_agg['lab_order_dttm'] <= icu_data_agg['after_24hr'])].reset_index(drop=True)

Lab_variables = [
   'albumin', 'alkaline_phosphatase',
       'ast', 'basophil', 'bilirubin_conjugated', 'bilirubin_total', 'calcium',
       'chloride', 'glucose_serum', 'hemoglobin', 'lymphocyte', 'monocyte',
       'neutrophil', 'platelet count', 'potassium', 'sodium', 'total_protein',
       'wbc'
]
agg_dict = {var: ['min', 'max', 'mean'] for var in Lab_variables}

icu_data_agg = icu_data_agg.groupby('encounter_id').agg(agg_dict).reset_index()

icu_data_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in icu_data_agg.columns.values]

In [ ]:
icu_data=pd.merge(icu_data,icu_data_agg, on=['encounter_id'], how='left')

#### model

In [ ]:
export=True

In [ ]:
test_data = icu_data[icu_data['admission_dttm'].dt.year.isin([2020, 2021])]
train_data = icu_data[~icu_data['admission_dttm'].dt.year.isin([2020, 2021])]

#train_data, test_data = train_test_split(icu_data, test_size=0.2, random_state=42)
# cutoff_date = icu_data['admission_dttm'].quantile(0.7)
# print(cutoff_date)
# train_data = icu_data[icu_data['admission_dttm'] <= cutoff_date]
# test_data = icu_data[icu_data['admission_dttm'] > cutoff_date]

model_col=['isfemale','age', 'min_bmi', 'max_bmi', 'avg_bmi',
       'min_weight_kg', 'max_weight_kg', 'avg_weight_kg', 'min_pulse',
       'max_pulse', 'avg_pulse', 'min_sbp', 'max_sbp', 'avg_sbp', 'min_dbp',
       'max_dbp', 'avg_dbp', 'min_temp_c', 'max_temp_c', 'avg_temp_c',
       'albumin_min', 'albumin_max', 'albumin_mean',
       'alkaline_phosphatase_min', 'alkaline_phosphatase_max',
       'alkaline_phosphatase_mean', 'ast_min', 'ast_max', 'ast_mean',
       'basophil_min', 'basophil_max', 'basophil_mean',
       'bilirubin_conjugated_min', 'bilirubin_conjugated_max',
       'bilirubin_conjugated_mean', 'bilirubin_total_min',
       'bilirubin_total_max', 'bilirubin_total_mean', 'calcium_min',
       'calcium_max', 'calcium_mean', 'chloride_min', 'chloride_max',
       'chloride_mean', 'glucose_serum_min', 'glucose_serum_max',
       'glucose_serum_mean', 'hemoglobin_min', 'hemoglobin_max',
       'hemoglobin_mean', 'lymphocyte_min', 'lymphocyte_max',
       'lymphocyte_mean', 'monocyte_min', 'monocyte_max', 'monocyte_mean',
       'neutrophil_min', 'neutrophil_max', 'neutrophil_mean',
       'platelet count_min', 'platelet count_max', 'platelet count_mean',
       'potassium_min', 'potassium_max', 'potassium_mean', 'sodium_min',
       'sodium_max', 'sodium_mean', 'total_protein_min', 'total_protein_max',
       'total_protein_mean', 'wbc_min', 'wbc_max', 'wbc_mean']

X_train=train_data[model_col]
y_train=train_data['isdeathdispo']

#test
X_test=test_data[model_col]
y_test=test_data['isdeathdispo']



In [ ]:
if export:
    train_data['Data_Type'] = 'train'
    test_data['Data_Type'] = 'test'

    # Concatenating the training and testing data into one DataFrame
    combined_data = pd.concat([train_data, test_data], ignore_index=True)
    combined_data.to_csv('train_test_dataset.csv', index=False)

In [ ]:
model = LGBMClassifier(
    boosting_type='gbdt',
    objective='binary',
    metric='binary_logloss',
    learning_rate=0.1,
    num_leaves=31,
    max_depth=10,
    n_estimators=50,
    feature_fraction= 1.0,
    device='gpu'  # Enable GPU acceleration
)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate metrics at default threshold (0.5)
default_threshold = 0.5
y_pred_default = (y_pred_proba >= default_threshold).astype(int)
accuracy = accuracy_score(y_test, y_pred_default)
recall = recall_score(y_test, y_pred_default)
precision = precision_score(y_test, y_pred_default)
roc_auc = roc_auc_score(y_test, y_pred_proba)
brier_score = brier_score_loss(y_test, y_pred_proba)

print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'ROC AUC: {roc_auc}')
print(f'brier_score : {brier_score}')

In [ ]:

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Create a subplot with 1 row and 2 columns
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot ROC curve on the first subplot
ax1.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
ax1.plot([0, 1], [0, 1], 'k--')  # random predictions curve
ax1.set_xlim([0.0, 1.0])
ax1.set_ylim([0.0, 1.05])
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate')
ax1.set_title('Receiver Operating Characteristic')
ax1.legend(loc="lower right")

# Calculate accuracy, recall, and precision at different thresholds for the second subplot
accuracies = []
recalls = []
precisions = []
for threshold in thresholds:
    y_pred_threshold = (y_pred_proba >= threshold).astype(int)
    accuracies.append(accuracy_score(y_test, y_pred_threshold))
    recalls.append(recall_score(y_test, y_pred_threshold))
    precisions.append(precision_score(y_test, y_pred_threshold))

# Plot accuracy, recall, and precision at different thresholds on the second subplot
ax2.plot(thresholds, accuracies, label='Accuracy')
ax2.plot(thresholds, recalls, label='Recall')
ax2.plot(thresholds, precisions, label='Precision')
ax2.set_xlim([0.0, 1.0])
ax2.set_ylim([0.0, 1.05])
ax2.set_xlabel('Threshold')
ax2.set_ylabel('Score')
ax2.set_title('Accuracy, Recall, and Precision at Different Thresholds')
ax2.legend(loc="lower left")

plt.tight_layout()
plt.show()


lgb.plot_importance(model, max_num_features=20, importance_type='split')
plt.title("Feature Importance (Split)")
plt.show()

lgb.plot_importance(model, max_num_features=20, importance_type='gain')
plt.title("Feature Importance (Gain)")
plt.show()

# creating a dataframe of target and probabilities
prob_df_lgbm = pd.DataFrame({'y':y_test, 'y_hat': y_pred_proba})

# binning the dataframe, so we can see success rates for bins of probability
bins = np.arange(0.05, 1.00, 0.05)
prob_df_lgbm.loc[:,'prob_bin'] = np.digitize(prob_df_lgbm['y_hat'], bins)
prob_df_lgbm.loc[:,'prob_bin_val'] = prob_df_lgbm['prob_bin'].replace(dict(zip(range(len(bins)), bins)))

# opening figure
plt.figure(figsize=(6,5), dpi=150)

# plotting ideal line
plt.plot([0,1],[0,1], 'k--', label='ideal')

# plotting calibration for lgbm
calibration_y = prob_df_lgbm.groupby('prob_bin_val')['y'].mean()
calibration_x = prob_df_lgbm.groupby('prob_bin_val')['y_hat'].mean()
plt.plot(calibration_x, calibration_y, marker='o', label='lgbm')

# legend and titles
plt.title('Calibration plot for LGBM')
plt.xlabel('Predicted probability')
plt.ylabel('Actual fraction of positives')
plt.legend()

def top_n_percentile(target_var, pred_proba):
    #thr_list = [0.99,0.97, 0.95,0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10]
    thr_list = np.arange(1, 0, -0.01)
    col = ['N Percentile', 'Thr Value','TN','FP','FN','TP','Sensitivity','Specificity','PPV', 'NPV' ,'Recall','Accuracy','site_name']
    result = pd.DataFrame(columns = col)
    i = 0
    
    for thr in thr_list: 
        prob = pd.DataFrame()
        prob['target_var'] = target_var
        prob['pred_proba'] = pred_proba

        thr_value = prob['pred_proba'].quantile(thr)
        prob['pred_proba_bin'] = np.where(prob['pred_proba'] >= thr_value, 1, 0)
        tn,fp,fn,tp = confusion_matrix(prob['target_var'], prob['pred_proba_bin']).ravel()

        sensitivity = tp/(tp+fn)
        specificity = tn/(tn+fp)
        ppv = tp/(tp+fp)
        npv = tn/(tn+fn)
        recall = tp/(tp+fn)
        acc = (tp+tn)/(tp+fn+tn+fp)
        n_prec = 'Top '+ str(np.round((1 - thr) * 100,0))+ "%"
        result.loc[i] = [n_prec,thr_value,tn,fp,fn,tp,sensitivity,specificity ,ppv,npv, recall, acc,f'{site_name}']
        i+=1
    return result
topn=top_n_percentile(y_test,y_pred_proba)
#topn.to_csv(f'Top_N_percentile_PPV_{site_name}.csv',index=False)

if export:
    timestamp = time.strftime("%Y%m%d-%H%M%S")
    model_filename = f"models/lgbm_model_{timestamp}.txt"
    model_filename_pkl = f"models/lgbm_model_{timestamp}.pkl"

    with open(model_filename_pkl, 'wb') as f:
        pickle.dump(model, f)
    
    model.booster_.save_model(model_filename)

    log_entry = f"Test AUC: {roc_auc}, Model: {model_filename}\n"

    with open("models/model_log.txt", "a") as log_file:
        log_file.write(log_entry)

    print(f"Model saved as {model_filename} and log updated.")

In [ ]:
# parameter search sample code
# import pandas as pd
# from sklearn.model_selection import GridSearchCV
# from lightgbm import LGBMClassifier
# from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, brier_score_loss

# # Define the parameter grid
# param_grid = {
#     'learning_rate': [0.01, 0.1, 0.2],
#     'num_leaves': [31, 50, 100],
#     'max_depth': [-1, 10, 20],
#     'n_estimators': [50, 100, 200],
#     'feature_fraction': [0.8, 0.9, 1.0]
# }

# # Initialize the model
# model = LGBMClassifier(
#     boosting_type='gbdt',
#     objective='binary',
#     metric='binary_logloss',
#     device='gpu'  # Enable GPU acceleration
# )

# # Perform grid search
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='roc_auc', verbose=2)
# grid_search.fit(X_train, y_train)

# # Collect results
# results = []
# for params, mean_score, scores in zip(grid_search.cv_results_['params'],
#                                       grid_search.cv_results_['mean_test_score'],
#                                       grid_search.cv_results_['std_test_score']):
#     print(f"Params: {params}, Mean ROC AUC: {mean_score:.4f}, Std: {scores:.4f}")
#     model.set_params(**params)
#     model.fit(X_train, y_train)
#     y_pred_proba = model.predict_proba(X_test)[:, 1]
#     y_pred_default = (y_pred_proba >= 0.5).astype(int)
#     accuracy = accuracy_score(y_test, y_pred_default)
#     recall = recall_score(y_test, y_pred_default)
#     precision = precision_score(y_test, y_pred_default)
#     roc_auc = roc_auc_score(y_test, y_pred_proba)
#     brier_score = brier_score_loss(y_test, y_pred_proba)
#     results.append({
#         'params': params,
#         'accuracy': accuracy,
#         'recall': recall,
#         'precision': precision,
#         'roc_auc': roc_auc,
#         'brier_score': brier_score
#     })

# # Convert results to DataFrame and save to CSV
# results_df = pd.DataFrame(results)
# results_df.to_csv('grid_search_results.csv', index=False)
# print(results_df)